In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from Modules.dataEngineering import dataEngineering
dataEngineering = dataEngineering()

In [2]:
df_ratings        = dataEngineering.loadRatings()['data']

In [3]:
from surprise import Dataset, KNNBasic, Reader
from surprise.model_selection import GridSearchCV

# Use movielens-100K
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['UserID', 'MovieID', 'userRating']], reader)

sim_options = {
    'name': ['cosine', 'pearson'],
    'user_based': [False, True],
}
bsl_options = {
    'method': ['sgd'],
    'learning_rate': [0.0005, 0.005]
}
param_grid = {
    'k' : [40, 60, 100],
    'sim_options': sim_options,
    'bsl_options': bsl_options
}

# สร้างตัว GridSearchCV สำหรับการค้นหาพารามิเตอร์
# gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=5)

# gs.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

In [4]:
# gs.best_score

{'rmse': 0.9577831002337381, 'mae': 0.7414442516505899}

{'rmse': 0.9577831002337381, 'mae': 0.7414442516505899}

In [5]:
# gs.best_params      

{'rmse': {'k': 100,
  'sim_options': {'name': 'pearson', 'user_based': False},
  'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}},
 'mae': {'k': 100,
  'sim_options': {'name': 'pearson', 'user_based': False},
  'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}}}

In [6]:
output = {'rmse': {'k': 100,
  'sim_options': {'name': 'pearson', 'user_based': False},
  'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}},
 'mae': {'k': 100,
  'sim_options': {'name': 'pearson', 'user_based': False},
  'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}}}

In [7]:
option_KNNBasic = {
    'k': 100,
    'sim_options': {'name': 'pearson', 'user_based': False},
    'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}
}

In [8]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [9]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

model_KNNBasic = KNNBasic(k=option_KNNBasic['k'], sim_options=option_KNNBasic['sim_options'], bsl_options=option_KNNBasic['bsl_options'],)

model_KNNBasic.fit(train_set)

predictions = model_KNNBasic.test(test_set)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9608
MAE:  0.7434


In [10]:
def predict_model(pivot):
    matrix = pivot.copy()
    u_col = 0
    ratings = []
    for u_id, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            m_id = matrix.columns[m_col]
            if pd.isnull(item):
                prep_rating = round(model_KNNBasic.predict(u_id, m_id).est, 4)
                ratings.append([u_id, m_id, prep_rating])
                matrix.iloc[u_col][m_id] = prep_rating
        u_col += 1
    
    recomment_unwatched = pd.DataFrame(ratings, columns=['UserID', 'MovieID', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)

In [11]:
df_pivot = df_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

In [12]:
mat, prep = predict_model(df_pivot.iloc[1:2])

In [13]:
mat

MovieID,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
UserID,,,,,,,,,,,,,,,,,,,,,
2,4.0008,4.0512,3.8213,3.0,4.35,3.9185,3.964,3.5018,3.57,3.8545,...,3.5018,3.5018,3.5018,3.5018,3.5018,3.5018,3.5018,3.5018,3.5018,3.5018


In [14]:
prep.head(10)

,UserID,MovieID,prep_Rating
7560,2,86548,5.0
7659,2,89759,5.0
8225,2,105211,5.0
9573,2,177763,5.0
6886,2,64197,5.0
9321,2,162478,5.0
7127,2,71579,5.0
8345,2,109850,5.0
6174,2,45175,5.0
9452,2,170813,5.0


Save Model

In [15]:
import joblib

joblib.dump(model_KNNBasic, 'models/model_KNNBasic.pkl')

['models/model_KNNBasic.pkl']